In [1]:
# --- 0. Instalación y Setup ---
!pip install roboflow ultralytics -q
from ultralytics import YOLO
import os
import yaml
from roboflow import Roboflow
from google.colab import drive
from IPython.display import clear_output, display
import pandas as pd
import numpy as np
import logging

# Desactiva los logs detallados de Ultralytics durante la validación
logging.getLogger('ultralytics').setLevel(logging.ERROR)

# --- 1. CONFIGURACIÓN PRINCIPAL ---
ROBOFLOW_API_KEY = "tZR7XZhFvySrbVM3hfH4"
ROBOFLOW_WORKSPACE = "luchini"
ROBOFLOW_PROJECT = "datasetenproceso-irzcf"
VALIDATION_DATASET_VERSION = 18

# 📌 RUTA ESPECÍFICA DE TU MODELO (Aseguramos que termine en .pt)
MODEL_PATH_BASE = '/content/drive/MyDrive/RepositorioTesisLuisCobos/ModelosConDatasetPropio/Coffee_Seg_n-Model_StableAug_720px_Batch16_v1n/weights/best'
MODEL_PATH = f"{MODEL_PATH_BASE}.pt"

print("Configuración cargada. Montando Google Drive...")
drive.mount('/content/drive')
print("✅ Drive Montado.")

# --- 2. Descargar Dataset de VALIDACIÓN (Una sola vez) ---
print(f"\nDescargando dataset de validación (v{VALIDATION_DATASET_VERSION}) desde Roboflow...")
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(ROBOFLOW_WORKSPACE).project(ROBOFLOW_PROJECT)
version = project.version(VALIDATION_DATASET_VERSION)
dataset = version.download("yolov8")

validation_data_yaml_path = f'{dataset.location}/data.yaml'

# Ajuste de rutas en el YAML
with open(validation_data_yaml_path, 'r') as f:
    data_yaml = yaml.safe_load(f)
data_yaml['train'] = f"{dataset.location}/train/images"
data_yaml['val'] = f"{dataset.location}/valid/images"
if os.path.exists(f"{dataset.location}/test/images"):
    data_yaml['test'] = f"{dataset.location}/test/images"
with open(validation_data_yaml_path, 'w') as f:
    yaml.dump(data_yaml, f)

clear_output(wait=True)
print("✅ Drive Montado y Dataset listo.")

# --- 3. Validación del Modelo Específico ---

if not os.path.exists(MODEL_PATH):
    print(f"❌ ERROR: No se encontró el modelo en: {MODEL_PATH}")
else:
    print(f"\n--- Evaluando Modelo: {os.path.basename(MODEL_PATH)} ---")

    # Cargar modelo
    model = YOLO(MODEL_PATH)

    # Ejecutar validación
    # Usamos batch=1 para asegurar máxima precisión en cálculo de métricas (o 16 si prefieres velocidad)
    metrics = model.val(
        data=validation_data_yaml_path,
        split='val',
        imgsz=640, # Ajusta a 720 si fue entrenado así, pero 640 es estándar
        batch=16,
        verbose=False,
        save=False
    )

    # --- 4. CONSTRUCCIÓN DE LA TABLA DETALLADA ---

    # Lista para guardar las filas de la tabla
    table_rows = []

    # 1. Fila "Todas" (Promedios globales)
    # Nota: Usamos las métricas de SEGMENTACIÓN (.seg)
    table_rows.append({
        "Clase": "Todas",
        "mAP@.5 (Mask)": metrics.seg.map50,
        "mAP@.5-.95 (Mask)": metrics.seg.map,
        "Precisión (P)": metrics.seg.mp, # Mean Precision
        "Sensibilidad (R)": metrics.seg.mr # Mean Recall
    })

    # 2. Filas por Clase individual
    # metrics.names es un diccionario {0: 'Maduro', 1: 'Verde'...}
    # metrics.seg.p es un array con la precisión por clase
    # metrics.seg.r es un array con el recall por clase
    # metrics.seg.maps es un array con el mAP50-95 por clase
    # metrics.seg.all_ap es una matriz (N_clases, 10). La columna 0 es mAP@.50

    class_indices = metrics.names.keys()

    for i in class_indices:
        class_name = metrics.names[i]

        # Extracción segura de datos
        p_class = metrics.seg.p[i]
        r_class = metrics.seg.r[i]
        map50_95_class = metrics.seg.maps[i]
        map50_class = metrics.seg.all_ap[i, 0] # La primera columna de all_ap corresponde al IoU 0.5

        table_rows.append({
            "Clase": class_name,
            "mAP@.5 (Mask)": map50_class,
            "mAP@.5-.95 (Mask)": map50_95_class,
            "Precisión (P)": p_class,
            "Sensibilidad (R)": r_class
        })

    # Crear DataFrame
    df_results = pd.DataFrame(table_rows)

    # Reordenar columnas según tu petición
    cols_order = ["Clase", "mAP@.5 (Mask)", "mAP@.5-.95 (Mask)", "Precisión (P)", "Sensibilidad (R)"]
    df_results = df_results[cols_order]

    # Formatear números a 3 decimales
    numeric_cols = ["mAP@.5 (Mask)", "mAP@.5-.95 (Mask)", "Precisión (P)", "Sensibilidad (R)"]
    for col in numeric_cols:
        df_results[col] = df_results[col].apply(lambda x: f"{x:.3f}")

    print("\n\n--- 🏆 Tabla III. Resultados del modelo en el conjunto de validación 🏆 ---")
    display(df_results)

    # Opcional: Guardar a Excel para copiar y pegar fácil
    # df_results.to_excel("Tabla_Resultados_Tesis.xlsx", index=False)

✅ Drive Montado y Dataset listo.

--- Evaluando Modelo: best.pt ---


--- 🏆 Tabla III. Resultados del modelo en el conjunto de validación 🏆 ---


,Clase,mAP@.5 (Mask),mAP@.5-.95 (Mask),Precisión (P),Sensibilidad (R)
0,Todas,0.879,0.764,0.815,0.812
1,maduro,0.859,0.752,0.766,0.790
2,pinton,0.915,0.823,0.876,0.826
3,seco,0.950,0.799,0.887,0.941
4,sobremaduro,0.769,0.672,0.752,0.618
5,verde,0.904,0.772,0.796,0.886
